In [ ]:
import os
import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from helpers.types import *
from helpers.data_prep import *
from helpers.visualization import *
from helpers.solar_conversions import *

In [ ]:
class LED_Lighting:
    def __init__(self, barrel_count):
        # VU-HORTI-BLADE-LMRR-2036-80W-CCHV-IP68
        self.energy_efficiency: mol_per_joule = 2.7e-6
        self.max_power_per_light: W = 80
        self.max_PPF: umol_per_s = 216
        self.price: EUR = 120
        self.efficiency = 0.85
        self.heat_per_driver: W = 20

        # Lighting config
        self.driver_count = 2
        self.barrel_count = barrel_count
        self.light_count_per_barrel = 3
        self.barrel_surface: m2 = 1.8 * 1.5

        # Derived properties
        self.max_PPFD: umol_per_m2_s = self.light_count_per_barrel * self.max_PPF / self.barrel_surface
        self.installed_power: W = self.barrel_count * self.light_count_per_barrel * self.max_power_per_light

        # Registers
        self.dimmer = []


    def run(self, timestamp, photoperiod, time_period: s, PPFD_to_supplement: umol_per_m2_s):
        total_PPFD_amount_to_supplement: mol = PPFD_to_supplement * time_period * (self.barrel_surface * self.barrel_count) / 1e6
        required_energy: J = total_PPFD_amount_to_supplement / self.energy_efficiency
        required_power: W = required_energy / time_period
        dimmer = required_power / self.installed_power

        heat_by_drivers: J = self.driver_count * self.heat_per_driver * dimmer * time_period
        heat_generated: J = (1 - self.efficiency) * required_energy + heat_by_drivers

        if required_power > self.installed_power:
            raise Exception(f"Light requirement ({round(required_power)} W) exceeds installed power available ({self.installed_power} W)")

        self.dimmer.append(dimmer)

        return required_energy, heat_generated


### Visualize

In [ ]:
light = LED_Lighting(barrel_count=6)

In [ ]:
if not os.environ.get("USING_RUN"):
    dli = 13
    photoperiod = 16
    roof_transparency = 0.4

    resample_period = "15min"
    time_period = pd.to_timedelta(resample_period).total_seconds()

    df = get_weather_data(date_from="2020-06-02", date_to="2020-06-03", resample_period=resample_period)

    for i, row in df.iterrows():
        irradiance: W_per_m2 = row["solarradiation"]
        natural_PPFD: umol_per_m2_s = irradiance_to_PPFD(irradiance)
        required_PPFD: umol_per_m2_s = projected_DLI_to_PPFD(dli=dli, photoperiod=photoperiod)

        PPFD_to_supplement: umol_per_m2_s = 0
        if required_PPFD > natural_PPFD:
            PPFD_to_supplement = required_PPFD - natural_PPFD

        required_energy, heat_generated = light.run(
            timestamp=i,
            photoperiod=photoperiod,
            time_period=time_period,
            PPFD_to_supplement=PPFD_to_supplement
        )

        df.loc[i, "lighting_power"] = required_energy / time_period
        df.loc[i, "heat_generated_W"] = heat_generated / time_period


In [ ]:
plot = None
if not os.environ.get("USING_RUN"):
    plot = plot_multiline(
        df, 
        ["lighting_power", "heat_generated_W"], 
        width=900, 
        height=235, 
        title="", 
        y_label="Watts", 
        legend_label="", 
        date_format=""
    )
plot

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>